In [ ]:
###############11111111111111111111111

import subprocess
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from moviepy.editor import VideoFileClip #this software requires the use of an earlier version moviepy-1.0.3
from dateutil import parser


# List of configurations for different participants, conditions, and hand types
configurations = [
    # {'participant': '047', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '047', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '047', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '052', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '052', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '052', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '053', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '053', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '053', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '055', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '055', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '055', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '056', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '056', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '056', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '058', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '058', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '058', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '058', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '059', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '059', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '059', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '046', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '064', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '064', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '065', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '065', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '065', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '066', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '066', 'condition': 'Ball', 'hand_type': 'RH'},
    # {'participant': '066', 'condition': 'Ball', 'hand_type': 'LH'},
    # {'participant': '067', 'condition': 'Ball', 'hand_type': 'BH'},
    # {'participant': '067', 'condition': 'Ball', 'hand_type': 'RH'},
    {'participant': '067', 'condition': 'Ball', 'hand_type': 'LH'}
]

# Function to extract metadata using ffprobe
def extract_metadata(video_file_path):
    data = {'Total Frames': None, 'Timecode': None, 'Creation Time': None, 'Frame Rate': None, 'Audio Sample Rate': None, 'File Path': video_file_path}
    video_metadata_command = (
        f'ffprobe -v error -select_streams v:0 -show_entries stream=nb_frames,r_frame_rate,duration -show_entries stream_tags=timecode -show_entries format_tags=creation_time -of default=noprint_wrappers=1 "{video_file_path}"'
    )
    audio_metadata_command = (
        f'ffprobe -v error -select_streams a:0 -show_entries stream=sample_rate -of default=noprint_wrappers=1 "{video_file_path}"'
    )
    video_metadata_process = subprocess.run(video_metadata_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    audio_metadata_process = subprocess.run(audio_metadata_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if video_metadata_process.returncode == 0:
        for line in video_metadata_process.stdout.split('\n'):
            if 'nb_frames' in line:
                _, value = line.split('=')
                data['Total Frames'] = value.strip() if value.strip().isdigit() else 'Calculation needed'
            elif 'TAG:timecode' in line:
                _, value = line.split('=')
                data['Timecode'] = value.strip()
            elif 'TAG:creation_time' in line:
                _, value = line.split('=')
                data['Creation Time'] = value.strip()
            elif 'r_frame_rate' in line:
                _, value = line.split('=')
                numerator, denominator = map(int, value.strip().split('/'))
                data['Frame Rate'] = str(numerator / denominator if denominator != 0 else numerator)
            if data['Creation Time'] and data['Timecode']:
                creation_datetime = datetime.strptime(data['Creation Time'], "%Y-%m-%dT%H:%M:%S.%fZ")
                time_parts = data['Timecode'].split(':')
                if len(time_parts) == 4:
                    fused_timecode = f"{creation_datetime.strftime('%Y-%m-%d')}T{creation_datetime.strftime('%H')}:{time_parts[1]}:{time_parts[2]}.{int(time_parts[3]):02d}Z"
                    data['Fused Timecode'] = fused_timecode
    else:
        print("Error extracting video metadata:", video_metadata_process.stderr)

    if audio_metadata_process.returncode == 0:
        for line in audio_metadata_process.stdout.split('\n'):
            if 'sample_rate' in line:
                _, value = line.split('=')
                data['Audio Sample Rate'] = value.strip()
    else:
        print("Error extracting audio metadata:", audio_metadata_process.stderr)

    return data

def inject_metadata(input_path, creation_time, timecode):
    output_path = input_path.replace("_synced.MP4", "_synced_timecode.MP4")
    command = [
        'ffmpeg',
        '-i', input_path,
        '-c', 'copy',
        '-metadata', f'creation_time={creation_time}',
        '-metadata', f'timecode={timecode}',
        output_path
    ]
    subprocess.run(command, check=True)
    print(f"Metadata injection completed for: {output_path}")

# Iterate over each configuration
for config in configurations:
    participant = config['participant']
    condition = config['condition']
    hand_type = config['hand_type']

    # # Define paths based on the configuration
    # video_paths = {
    #     "Results": f"/Volumes/Samsung_T5/Experiment Data/SDC_{participant}/Videos/Results/SDC_{participant}_{condition}_{hand_type}_R.MP4",
    #     "VideopPanel": f"/Volumes/Samsung_T5/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"
    # }

        # Define paths based on the configuration
    # Define paths based on the configuration
    video_paths = {
        "Results": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Results/SDC_{participant}_{condition}_{hand_type}_R.MP4",
        "VideopPanel": f"/Volumes/Beorn_4T/Experiment Data/SDC_{participant}/Videos/Videopanel/SDC_{participant}_{condition}_{hand_type}_VP.MP4"    
        }
    video_file_paths = [video_paths["Results"], video_paths["VideopPanel"]]

    # Extract metadata and store it in a DataFrame
    metadata_list = [extract_metadata(path) for path in video_file_paths]
    df = pd.DataFrame(metadata_list)


    # Convert 'Total Frames' and 'Frame Rate' to numeric values safely
    df['Total Frames'] = pd.to_numeric(df['Total Frames'], errors='coerce')
    df['Frame Rate'] = pd.to_numeric(df['Frame Rate'], errors='coerce')

    # Try to convert 'Fused Timecode' to datetime and handle potential errors
    try:
        df['Fused Timecode'] = df['Fused Timecode'].apply(lambda x: pd.to_datetime(x, errors='coerce'))
    except ValueError as e:
        print(f"Error converting Fused Timecode to datetime: {e}")
        # Handle or log error appropriately, perhaps skip this loop iteration with 'continue'
        continue

    # Check for NaNs which might result from conversion issues
    if df['Fused Timecode'].isna().any() or df['Total Frames'].isna().any() or df['Frame Rate'].isna().any():
        print("Warning: Missing or incorrect metadata values detected. Check the integrity of your video files' metadata.")
        continue  # Optionally skip processing this configuration if crucial data is missing

    # Compute the maximum start time only after ensuring the data integrity
    max_start_time = df['Fused Timecode'].max()
    df['End Time'] = df.apply(lambda row: row['Fused Timecode'] + pd.to_timedelta(row['Total Frames'] / row['Frame Rate'], unit='s') if not pd.isna(row['Total Frames']) and not pd.isna(row['Frame Rate']) else pd.NaT, axis=1)

    # Handle cases where 'End Time' could not be calculated due to missing data
    if df['End Time'].isna().any():
        print("Error: Unable to calculate 'End Time' due to missing data. This configuration will be skipped.")
        continue  # Skip to next configuration

    min_end_time = df['End Time'].min()
    duration = (min_end_time - max_start_time).total_seconds()

    for index, path in enumerate(video_file_paths):
        video = VideoFileClip(path)
        video_start_time = df.loc[index, 'Fused Timecode']
        start_time = (max_start_time - video_start_time).total_seconds()
        new_clip = video.subclip(max(0, start_time), max(0, start_time) + duration)
        new_filename = path.replace(".MP4", "_synced.MP4")
        new_clip.write_videofile(new_filename, codec='libx264')

    # Inject metadata into synchronized videos
    video_file_paths_synced = [path.replace(".MP4", "_synced.MP4") for path in video_file_paths]
    video_start_time = max_start_time  # Assuming this is the common synchronization time for metadata injection
    formatted_video_start_time = video_start_time.isoformat(timespec='milliseconds')
    hundredths_of_seconds = video_start_time.microsecond // 10000
    timecode_str = video_start_time.strftime("%H:%M:%S:") + f"{hundredths_of_seconds:02d}"

    for path in video_file_paths_synced:
        inject_metadata(path, formatted_video_start_time, timecode_str)

    # Check and print final metadata to ensure synchronization and metadata consistency
    final_metadata_list = [extract_metadata(path.replace(".MP4", "_timecode.MP4")) for path in video_file_paths_synced]
    final_df = pd.DataFrame(final_metadata_list)
    print(final_df)

# Note: You need to define `inject_metadata` function, which is used in the script for metadata injection.
